In [2]:
import pandas as pd
import string
import numpy as np

In [16]:
#%%
import os
print(os.getcwd())

d:\OneDrive - MNSCU\Research\2023_AI_Text_Classification\AI_Generated_Text_Classfier\code\classifier\ml_models


In [5]:
df1 = pd.read_csv('../../../data/features_GT.csv')
df1['Label'] = 0 #ground truth - human text

Index(['Unnamed: 0', 'word_count', 'char_count', 'word_density',
       'punctuation_count', 'title_word_count', 'upper_case_word_count',
       'noun_count', 'verb_count', 'adj_count',
       ...
       'lda10', 'lda11', 'lda12', 'lda13', 'lda14', 'lda15', 'lda16', 'lda17',
       'lda18', 'lda19'],
      dtype='object', length=50783)

In [7]:
df2 = pd.read_csv('../../../data/features_ChatGPT.csv')
df2['Label'] = 1 #GPT response

In [8]:
merged_df = pd.concat([df1, df2], ignore_index = True, sort = False)
print(merged_df.shape)
print(merged_df.columns)

(8534, 50784)
Index(['Unnamed: 0', 'word_count', 'char_count', 'word_density',
       'punctuation_count', 'title_word_count', 'upper_case_word_count',
       'noun_count', 'verb_count', 'adj_count',
       ...
       'lda11', 'lda12', 'lda13', 'lda14', 'lda15', 'lda16', 'lda17', 'lda18',
       'lda19', 'Label'],
      dtype='object', length=50784)


In [10]:
X = merged_df.iloc[:, 1:50783]
y = merged_df['Label']
print(X.shape, y.shape)

(8534, 50782) (8534,)


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = pd.DataFrame(X_train)
X_train = X_train.reset_index(drop=True)
X_test = pd.DataFrame(X_test)
X_test = X_test.reset_index(drop=True)
y_train = pd.DataFrame(y_train)
y_train = y_train.reset_index(drop=True)
y_test = pd.DataFrame(y_test)
y_test = y_test.reset_index(drop=True)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(5717, 50782) (5717, 1) (2817, 50782) (2817, 1)


In [13]:
import pickle
from sklearn.metrics import f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
#train random forest
rf_clf = RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=None, \
                                min_samples_split=10, min_samples_leaf=10, min_weight_fraction_leaf=0.0, \
                                max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, \
                                bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, \
                                warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

print("Random Forest Model fitting started ....")
rf_model = rf_clf.fit(X_train, y_train)
pickle.dump(rf_model, open('../../../model/wiki_rf.pkl', 'wb'))
print('RF Model saved')
y_pred = rf_clf.predict(X_test)
y_pred = np.where(y_pred <= 0.5, 0, 1)
f1 = f1_score(y_test, y_pred)
print('Rf - F1 Score = ', f1)
print(classification_report(y_test, y_pred))

Model fitting started ....


c:\Users\nttru\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == "":


Model saved
F1 Score =  0.998998998998999
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1317
           1       1.00      1.00      1.00      1500

    accuracy                           1.00      2817
   macro avg       1.00      1.00      1.00      2817
weighted avg       1.00      1.00      1.00      2817



In [14]:
#train SVM model
from sklearn import svm
from sklearn.metrics import f1_score, classification_report

svm_clf = svm.SVC(C=4.0, kernel='rbf', degree=7, gamma='scale', coef0=0.0, shrinking=True, \
                    probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, \
                    max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)

print("SVM Model fitting started ....")
svm_model = svm_clf.fit(X_train, y_train)
pickle.dump(svm_model, open('../../../model/wiki_svm.pkl', 'wb'))
print('SVM Model saved')
y_pred = svm_clf.predict(X_test)
y_pred = np.where(y_pred <= 0.5, 0, 1)
f1 = f1_score(y_test, y_pred)
print('SVM - F1 Score = ', f1)
print(classification_report(y_test, y_pred))

SVM Model fitting started ....


c:\Users\nttru\anaconda3\envs\py37\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Model saved
F1 Score =  0.7560819462227913
              precision    recall  f1-score   support

           0       0.73      0.66      0.70      1317
           1       0.73      0.79      0.76      1500

    accuracy                           0.73      2817
   macro avg       0.73      0.73      0.73      2817
weighted avg       0.73      0.73      0.73      2817



In [15]:
#XGB model
#import shap
from sklearn.metrics import f1_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier

xgb_clf = GradientBoostingClassifier(learning_rate=0.01, n_estimators=1000)
print("XGB Model fitting started ....")
model = xgb_clf.fit(X_train, y_train)
pickle.dump(model, open('../../../model/wiki_xgb.pkl', 'wb'))
print('Model saved')
y_pred = xgb_clf.predict(X_test)
y_pred = np.where(y_pred <= 0.5, 0, 1)
f1 = f1_score(y_test, y_pred)
print('F1 Score = ', f1)
print(classification_report(y_test, y_pred))
'''
explainer = shap.Explainer(model)
X_train = pd.DataFrame(X_train)
shap_values = explainer(X_train)
# visualize the first prediction's explanation
shap.plots.waterfall(shap_values[0])
'''

XGB Model fitting started ....


c:\Users\nttru\anaconda3\envs\py37\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Model saved
F1 Score =  0.9996665555185061
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1317
           1       1.00      1.00      1.00      1500

    accuracy                           1.00      2817
   macro avg       1.00      1.00      1.00      2817
weighted avg       1.00      1.00      1.00      2817



"\nexplainer = shap.Explainer(model)\nX_train = pd.DataFrame(X_train)\nshap_values = explainer(X_train)\n# visualize the first prediction's explanation\nshap.plots.waterfall(shap_values[0])\n"